### Gutensberg Books Web-Scrapping

##### Author: Kevin Okiah

**4/9/2019**

This note book is used to scrape GutenBerg books from **Childrens Section Book Shelf** http://www.gutenberg.org/wiki/Children%27s_Instructional_Books_(Bookshelf)  

GUTENBERG Header,Footer and Contents sections are stripped from each book programmatically.

Topic modeling will be performed on this books on a separate notebook.


In [1]:
import numpy as np
import pandas as pd
import selenium
from lxml import html
import urllib3
from bs4 import BeautifulSoup
import urllib 
import string
from urllib import request
from string import punctuation

import pickle
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

http = urllib3.PoolManager()

def get_url_Bs(url):
    tree = BeautifulSoup(url)
    return tree

def get_url_Sel(url):
    tree = html.document_fromstring(url)
    return tree

In [2]:
myurl = http.request('GET', "http://www.gutenberg.org/wiki/Children%27s_Instructional_Books_(Bookshelf)").data#http.request('GET', "https://www.brightermonday.co.ke/search/jobs-in-nairobi/").data#http.request('GET', "http://www.jobwebkenya.com").data #desired url

tree = get_url_Sel(myurl)

In [3]:
links = tree.xpath('//a/@href')

In [4]:
def Book_Titles():
    '''
    Function to scrap book titles
    '''
    categories= tree.xpath('//*[(@id = "mw-content-text")]//*[(((count(preceding-sibling::*)+ 1) = 1) and parent::*)]')
    temp =[]
    temp2 =[]
    for i in categories:
        temp = temp+ i.cssselect('a')
    for j in temp:
            #print(len([j.text_content().split()]))
            t =j.text_content()
            temp2 = temp2+ [t.replace('\n','')]
    temp3 =pd.unique(temp2)
    book_titles = [x for x in temp3 if x]
    book_titles =book_titles[13:]
    return(book_titles)

book_titles = Book_Titles()
book_titles.insert(53, 'Dutch')

In [5]:
len(book_titles)

104

In [6]:
#pull all hyperlinks
def Books_Urls():
    '''
    Function to pull books urls. Some books numbers listed below have unique 
    url setup as such reconstructing their addess had to be persomed as in the code below
    '''
    
    SpecialUrls = [23424, 13853, 22795, 18274, 21973, 15950, 24222, 21783,2441, 17160, 22917, 23395]
    
    Bookslinks = tree.xpath('//a/@href')
    Books_urls= {} #declare urls Dictonary 
    Books2_urls= {} #declare urls Dictonary 
    n =0
    for i in Bookslinks :
        T = str.split(i, '/')

        if "www.gutenberg.org" in T and 'http:' not in T: 
            #print(SpecialUrls)
            if(int(T[4]) in SpecialUrls):
                #Format 1: http://www.gutenberg.org/files/23424/23424-0.txt
                Books2_urls.update({n:"http://" + T[2]+"/files/"+T[4]+"/"+T[4]+"-0.txt" })
            else:
                #Format 2:  http://www.gutenberg.org/cache/epub/7841/pg7841.txt 
                Books2_urls.update({n:"http://" + T[2]+"/cache/epub/"+T[4]+"/pg"+T[4]+".txt" })
                #Format 3: http://www.gutenberg.org/files/23424/23424-0.txt

            n =n+1
    return(Books2_urls)
booksUrls = Books_Urls()

In [7]:
book_titles

['A Primary Reader: Old-time Stories, Fairy Tales and Myths Retold by Children',
 'The Bird-Woman of the Lewis and Clark Expedition',
 "Dr. Scudder's Tales for Little Readers, About the Heathen.",
 'The Louisa Alcott Reader: a Supplementary Reader for the Fourth Year of School',
 'Boy Blue and his friends, School ed.',
 'The Book of Nature Myths',
 "The Flag of My Country. Shikéyah Bidah Na'at'a'í;Navajo New World Readers 2",
 "Chambers's Elementary Science Readers, Book I",
 'The Little Lame Prince;Rewritten for Young Readers by Margaret Waters',
 "Harry's Ladder to Learning",
 'Little Present',
 'The Beacon Second Reader',
 "The Child's World Third Reader",
 'De La Salle Fifth Reader',
 'The Elson Readers, Book 5',
 'The Literary World Seventh Reader',
 "McGuffey's First Eclectic Reader, Revised Edition",
 "McGuffey's Second Eclectic Reader",
 "McGuffey's Third Eclectic Reader",
 "McGuffey's Fourth Eclectic Reader",
 "McGuffey's Fifth Eclectic Reader",
 "McGuffey's Sixth Eclectic Rea

In [8]:
def Book_CategoryMapper():
    '''
    Function tagging unique book categories. and mapping them to book urls
    '''
    
    titles_catagories = tree.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "mw-headline", " " ))]')
    tempx =[]
    BooksMapped = pd.DataFrame()
    for j in titles_catagories:
            tempx = tempx + [j.text_content()]
    Category_list =[]
    for i in tempx:
        if i =='Misc.':
            for n in range(11):
                Category_list = Category_list+ ['Misc.']
        if i =='Graded Readers':
            for n in range(20):
                Category_list = Category_list+ ['Graded Readers']
        if i =='Poetry Readers':
            for n in range(4):
                Category_list = Category_list+ ['Poetry Readers']
        if i =='Readers for English Speakers Learning Other Languages':
            for n in range(4):
                Category_list = Category_list+ ['Readers for English Speakers Learning Other Languages']
        if i =='Non-English Readers':
            for n in range(2):
                Category_list = Category_list+ ['Non-English Readers']
        if i =='About Readers':
            for n in range(1):
                Category_list = Category_list+ ['About Readers']
        if i =='Science and Nature':
            for n in range(29):
                Category_list = Category_list+ ['Science and Nature']
        if i =='History':
            for n in range(4):
                Category_list = Category_list+ ['History']
        if i =='Geography':
            for n in range(6):
                Category_list = Category_list+ ['Geography']
        if i =='Uncategorized':
            for n in range(23):
                Category_list = Category_list+ ['Uncategorized']
    BooksMapped["BookTitle"] =book_titles
    BooksMapped["Category"] = Category_list
    BooksMapped["url"] =booksUrls.values()
    return(BooksMapped)

In [9]:
BooksMapper =Book_CategoryMapper()

In [10]:
BooksMapper.head()

,BookTitle,Category,url
0,"A Primary Reader: Old-time Stories, Fairy Tale...",Misc.,http://www.gutenberg.org/cache/epub/7841/pg784...
1,The Bird-Woman of the Lewis and Clark Expedition,Misc.,http://www.gutenberg.org/cache/epub/5742/pg574...
2,"Dr. Scudder's Tales for Little Readers, About ...",Misc.,http://www.gutenberg.org/cache/epub/13539/pg13...
3,The Louisa Alcott Reader: a Supplementary Read...,Misc.,http://www.gutenberg.org/cache/epub/7425/pg742...
4,"Boy Blue and his friends, School ed.",Misc.,http://www.gutenberg.org/cache/epub/16046/pg16...


In [11]:
def bookstart(strT):
    '''
    Marker for beginning of books. There are ~20 unique ways the books starte
    '''
    possible_starts = ['CONTENTS.', 'CONTENTS', '~CONTENTS~', 
                       '[Illustration]', '[Illustration: ]', 
                       'ILLUSTRATIONS.', 'THE ALPHABET.',
                       'Lessons.', 'SELECTIONS IN PROSE AND POETRY.', 'CONTENTS (5)', "L'IMPRUDENCE.",
                      'POEMS TEACHERS ASK FOR', 'Contents.', 'HOW TO USE THE BOOK', 'STORIES FROM LIVY.', 
                       'TABLE DES MATIÈRES.', 'OUR COMMON INSECTS.', 'CHAPTER ONE', '[Contents of the Books]',
                      '[Illustration: Laplander Travelling.]','CHAPTER ONE.', 'Contents', 'Stories in This Book',
                      'INHOUD.','INTRODUCTION.', '_THE PLAN BOOK SERIES_', 'Stanford Achievement Test']
    
    return(strT.lstrip() in possible_starts)

def bookend(strT):
    '''
    Marker for end of book.
    '''
    possible_ends  = ['End of the Project Gutenberg EBook', "End of Project Gutenberg's"]
    
    return(strT.startswith(possible_ends[0]) or strT.startswith(possible_ends[1]))

In [12]:
def parseBook(key =0,url="http://www.gutenberg.org/files/2441/2441-0.txt"):
    '''
    Strip header and footer GUTENBERG  from a book
    '''
    CleanBook =""
    Title =""
    Author =""
    try:
        respose = request.urlopen(url)
        raw = respose.read().decode("utf-8")
        #table = str.maketrans('', '', string.punctuation)
        Temp = list(raw.splitlines()) #Split raw the test in raw and store in an array
        flag = 0 #flag to track when to read and close a book
        CleanBook = [] #array to hold the desired sentences
        CleanTokens = []
        Author =""
        Title =""
        Tokens =[]
        for i in Temp:
            if(bookstart(i)==True):
                flag =1
            if(bookend(i)==True):
                flag =2
            try:
                Title = book_titles[key]
            except IndexError as e:
                print(e.args)
            Author = "tbd"
            if(flag==1):
                if(len(i)>0):
                    CleanBook.append(i)

    except request.HTTPError as err:
        if err.code == 404:
            print(err.code, " Error!!!")
        else:
            raise
    except UnicodeDecodeError  as e:
        print("UnicodeDecodeError!!  ", e.reason)
        #raise
    
    Book ={}
    
    Book = {"Title":Title, "Body":[CleanBook]}
    return(Book)



Book = parseBook()

In [13]:
CleanBooks = {}
temp = pd.DataFrame()
for key, value in booksUrls.items():
    try:
        Book = parseBook(key, value)
        print ("processing :", key, value )#, 
        CleanBooks.update({Book['Title']:Book['Body']})
    except IndexError as e:
        print(e.args)
print("processing complete...")

processing : 0 http://www.gutenberg.org/cache/epub/7841/pg7841.txt
processing : 1 http://www.gutenberg.org/cache/epub/5742/pg5742.txt
processing : 2 http://www.gutenberg.org/cache/epub/13539/pg13539.txt
processing : 3 http://www.gutenberg.org/cache/epub/7425/pg7425.txt
processing : 4 http://www.gutenberg.org/cache/epub/16046/pg16046.txt
processing : 5 http://www.gutenberg.org/cache/epub/22420/pg22420.txt
processing : 6 http://www.gutenberg.org/files/23424/23424-0.txt
processing : 7 http://www.gutenberg.org/cache/epub/18217/pg18217.txt
processing : 8 http://www.gutenberg.org/cache/epub/24053/pg24053.txt
processing : 9 http://www.gutenberg.org/cache/epub/24644/pg24644.txt
processing : 10 http://www.gutenberg.org/cache/epub/24703/pg24703.txt
processing : 11 http://www.gutenberg.org/cache/epub/15659/pg15659.txt
processing : 12 http://www.gutenberg.org/cache/epub/15170/pg15170.txt
processing : 13 http://www.gutenberg.org/cache/epub/10811/pg10811.txt
processing : 14 http://www.gutenberg.org/

In [14]:
temp = pd.DataFrame.from_dict(CleanBooks).T
temp = temp.reset_index()
temp.columns = ['BookTitle','Body']
temp.head()

,BookTitle,Body
0,"A Primary Reader: Old-time Stories, Fairy Tale...","[CONTENTS., THE UGLY DUCKLING, THE LITTLE PINE..."
1,The Bird-Woman of the Lewis and Clark Expedition,"[CONTENTS, THE BIRD-WOMAN, WHO THE WHITE MEN W..."
2,"Dr. Scudder's Tales for Little Readers, About ...","[CONTENTS., CHAPTER I., General Remarks, CHAPT..."
3,The Louisa Alcott Reader: a Supplementary Read...,"[CONTENTS., I. A CHRISTMAS DREAM, II. THE CAND..."
4,"Boy Blue and his friends, School ed.","[~CONTENTS~, LITTLE BOY BLUE, SNOWBALL, FIRE-C..."


In [15]:
BooksMapper.head()

,BookTitle,Category,url
0,"A Primary Reader: Old-time Stories, Fairy Tale...",Misc.,http://www.gutenberg.org/cache/epub/7841/pg784...
1,The Bird-Woman of the Lewis and Clark Expedition,Misc.,http://www.gutenberg.org/cache/epub/5742/pg574...
2,"Dr. Scudder's Tales for Little Readers, About ...",Misc.,http://www.gutenberg.org/cache/epub/13539/pg13...
3,The Louisa Alcott Reader: a Supplementary Read...,Misc.,http://www.gutenberg.org/cache/epub/7425/pg742...
4,"Boy Blue and his friends, School ed.",Misc.,http://www.gutenberg.org/cache/epub/16046/pg16...


In [16]:
GuternsbergBooks = pd.merge(BooksMapper, temp, on='BookTitle', how='outer')

In [17]:
GuternsbergBooks.head()

,BookTitle,Category,url,Body
0,"A Primary Reader: Old-time Stories, Fairy Tale...",Misc.,http://www.gutenberg.org/cache/epub/7841/pg784...,"[CONTENTS., THE UGLY DUCKLING, THE LITTLE PINE..."
1,The Bird-Woman of the Lewis and Clark Expedition,Misc.,http://www.gutenberg.org/cache/epub/5742/pg574...,"[CONTENTS, THE BIRD-WOMAN, WHO THE WHITE MEN W..."
2,"Dr. Scudder's Tales for Little Readers, About ...",Misc.,http://www.gutenberg.org/cache/epub/13539/pg13...,"[CONTENTS., CHAPTER I., General Remarks, CHAPT..."
3,The Louisa Alcott Reader: a Supplementary Read...,Misc.,http://www.gutenberg.org/cache/epub/7425/pg742...,"[CONTENTS., I. A CHRISTMAS DREAM, II. THE CAND..."
4,"Boy Blue and his friends, School ed.",Misc.,http://www.gutenberg.org/cache/epub/16046/pg16...,"[~CONTENTS~, LITTLE BOY BLUE, SNOWBALL, FIRE-C..."


In [18]:
import os
from unipath import Path
wd = os.getcwd()
p = Path(wd)
path = str(p.parent)

In [19]:
# Saving Data for future analysis
GuternsbergBooks.to_csv(path+'/Data/GuternsbergBooksRaw.csv', header=True, index=False, encoding='utf-8')

In [20]:
%load_ext version_information
%version_information pandas, numpy, requests, bs4, selenium, lxml, urllib3, pyvirtualdisplay, unipath

Software,Version
Python,3.6.8 64bit [GCC 7.3.0]
IPython,7.2.0
OS,Linux 4.15.0 46 generic x86_64 with debian buster sid
pandas,0.24.2
numpy,1.16.2
requests,2.21.0
bs4,4.7.1
selenium,3.141.0
lxml,4.3.2
urllib3,1.24.1
